In [1]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle
from wordcloud import WordCloud

In [16]:
part1 = pd.read_csv("full_texts_to_label_full4.csv")
part2 = pd.read_csv("full_texts_to_label_random1.csv")
part3 = pd.read_csv("full_texts_to_label_random2.csv")
part4 = pd.read_csv("full_texts_to_label_random3.csv")
part5 = pd.read_csv("full_texts_to_label_full_previous.csv")
full_label_texts = pd.concat([part1, part2, part3, part4, part5])
full_label_processed = add_processed_text(full_label_texts, "text")

In [2]:
full_labels = pd.read_csv('../input_data/full_datasets/fully_labelled_texts.csv')

In [15]:
full_labels_full_processed = add_processed_text(full_labels, 'content')

In [17]:
full_labels_full = full_labels_full_processed[~full_labels_full_processed.processed_text.isin(full_label_processed['processed_text'])]

In [20]:
full_labels_full = full_labels_full.drop(['processed_text'], axis=1)

In [21]:
import pickle
smoking_model = pickle.load(open('../data_exploration/best_models/best_smoking_ex1.pkl', 'rb'))
drinking_model = pickle.load(open('../data_exploration/best_models/best_drinking_ex1.pkl', 'rb'))
drugs_model = pickle.load(open('../data_exploration/best_models/best_drugs_ex1.pkl', 'rb'))

In [22]:
stemmer = SnowballStemmer("dutch")
np.random.seed(500)

def create_preprocess_corpus(input_corpus, content_name, label_name, smoking=False, second_experiment=False, skip_rows=[]):
    corpus = input_corpus[['Unnamed: 0', content_name, label_name]]
    corpus = corpus.rename({content_name:'text', label_name: 'label'}, axis=1)
    corpus['processed_text'] = corpus['text'].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    if second_experiment:
        if smoking:
            replace_text = 'Rookt niet'
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Rookt niet')
            corpus['label'] = corpus['label'].str.replace('Rookte', 'Rookt niet')
        else:
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Nee')
        
    corpus = corpus.drop(corpus[corpus.label == '--'].index)
    corpus = corpus.drop(corpus[corpus.label == 'Onbekend'].index)
    corpus = corpus[~corpus.processed_text.str.contains('vertrouwelijk')]
    corpus_backup = corpus.copy()
    return corpus, corpus_backup

def add_processed_text(input_corpus, text_column):
    corpus = input_corpus.copy()
    corpus['processed_text'] = corpus[text_column].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    return corpus

In [23]:
smoking_corpus, _ = create_preprocess_corpus(full_labels_full, 'content', 'roken_answer_label', smoking=True)
drinking_corpus, _ = create_preprocess_corpus(full_labels_full, 'content', 'alcohol_answer_label')
drugs_corpus, _ = create_preprocess_corpus(full_labels_full, 'content', 'drugs_answer_label')

In [24]:
smoking_corpus

,Unnamed: 0,text,label,processed_text
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,"indicatie: x-bekken, x-heup axiaal klinishe ..."
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,"anamnese: geen klachten van pijn op de borst,..."
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,   [initials] [lastname] [streetname] e...
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,"geachte collega, bovengenoemde patient was op..."
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,"geachte collega, bovengenoemde patiente was o..."
...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,beloop: beloop: tc op verzoek patient beleid...
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,anamnese: verkort consult: voorgeschiedenis ...
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,beloop: beloop: zie mri lang traject 7 cm ext...
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,samenvatting: voorgeschiedenis: blanco. medic...


In [25]:



drinking_corpus

,Unnamed: 0,text,label,processed_text
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,"indicatie: x-bekken, x-heup axiaal klinishe ..."
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,"anamnese: geen klachten van pijn op de borst,..."
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,   [initials] [lastname] [streetname] e...
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,"geachte collega, bovengenoemde patient was op..."
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,"geachte collega, bovengenoemde patiente was o..."
...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,beloop: beloop: tc op verzoek patient beleid...
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Ja,anamnese: verkort consult: voorgeschiedenis ...
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,beloop: beloop: zie mri lang traject 7 cm ext...
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,samenvatting: voorgeschiedenis: blanco. medic...


In [26]:
_, smoking_x_test, _, smoking_y_test = train_test_split(smoking_corpus['processed_text'], smoking_corpus['label'], test_size=0.2, random_state=50)
_, drinking_x_test, _, drinking_y_test = train_test_split(drinking_corpus['processed_text'], drinking_corpus['label'], test_size=0.2, random_state=50)
_, drugs_x_test, _, drugs_y_test = train_test_split(drugs_corpus['processed_text'], drugs_corpus['label'], test_size=0.2, random_state=50)

# Smoking

In [31]:
predicted_nb_df = smoking_model.decision_function(smoking_x_test)
predicted_nb_labels = smoking_model.predict(smoking_x_test)
x_test_probs = pd.DataFrame(smoking_x_test, columns=['processed_text'])
x_test_probs['nothing_found_eval'] = [x[0] for x in predicted_nb_df]
x_test_probs['current_user_eval'] = [x[1] for x in predicted_nb_df]
x_test_probs['non_user_eval'] = [x[2] for x in predicted_nb_df]
x_test_probs['previous_user_eval'] = [x[3] for x in predicted_nb_df]
x_test_probs['predicted_label'] = predicted_nb_labels
x_test_probs['roken_answer_label'] = smoking_y_test
x_test_probs = x_test_probs.merge(smoking_corpus[['text', 'processed_text']], left_on='processed_text', right_on='processed_text', how='left')

In [32]:
sorted_probs_current = x_test_probs.sort_values(by=['current_user_eval'], ascending=False)
sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)].sort_values(by=['non_user_eval'], ascending=False)
sorted_probs_previous = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][~x_test_probs.processed_text.isin(sorted_probs_non.head(100).processed_text)].sort_values(by=['previous_user_eval'], ascending=False)
texts_to_label = sorted_probs_current[['text', 'processed_text', 'roken_answer_label']].head(100)
texts_to_label = texts_to_label.append(sorted_probs_non[~sorted_probs_non.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'roken_answer_label']].head(100))
texts_to_label = texts_to_label.append(sorted_probs_previous[~sorted_probs_previous.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'roken_answer_label']].head(100))
texts_to_label = texts_to_label.merge(drinking_corpus.rename(columns={'label': 'alcohol_answer_label'})[['processed_text', 'alcohol_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label = texts_to_label.merge(drugs_corpus.rename(columns={'label': 'drugs_answer_label'})[['processed_text', 'drugs_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label = texts_to_label.drop_duplicates()

/tmp/ipykernel_4182246/2220941128.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sorted_probs_previous = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][~x_test_probs.processed_text.isin(sorted_probs_non.head(100).processed_text)].sort_values(by=['previous_user_eval'], ascending=False)
/tmp/ipykernel_4182246/2220941128.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label = texts_to_label.append(sorted_probs_non[~sorted_probs_non.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'roken_answer_label']].head(100))
/tmp/ipykernel_4182246/2220941128.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label = texts_to_label.append(sorted_probs_previous[~sorted_probs_previous.processe

In [33]:
texts_to_label

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Samenvatting: \tVoorgeschiedenis: pleiomorf ad...,samenvatting: voorgeschiedenis: pleiomorf ade...,Rookt,Niets gevonden,Niets gevonden
1,Anamnese: \t Verkort consult: Voorgeschiedenis...,anamnese: verkort consult: voorgeschiedenis:...,Rookt,Ja,Nee
2,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
3,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
4,Anamnese: \t Verkort consult: TC: Aandachtspun...,anamnese: verkort consult: tc: aandachtspunt...,Rookt,Nee,Nee
...,...,...,...,...,...
295,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
296,Anamnese: \tIndicatie KJD uit + doorbewegen k...,anamnese: indicatie kjd uit + doorbewegen kn...,Rookte,Nee,Nee
297,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Ja,Nee
298,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Ja,Nee


# Drinking

In [34]:
predicted_nb_df = drinking_model.decision_function(drinking_x_test)
predicted_nb = drinking_model.predict(drinking_x_test)
x_test_probs = pd.DataFrame(drinking_x_test, columns=['processed_text'])
x_test_probs['current_user_eval'] = [x[0] for x in predicted_nb_df]
x_test_probs['non_user_eval'] = [x[1] for x in predicted_nb_df]
x_test_probs['nothing_found_eval'] = [x[2] for x in predicted_nb_df]
x_test_probs['predicted_label'] = predicted_nb
x_test_probs['alcohol_answer_label'] = drinking_y_test
x_test_probs = x_test_probs.merge(drinking_corpus[['text', 'processed_text']], left_on='processed_text', right_on='processed_text', how='left')

In [35]:
sorted_probs_current = x_test_probs.sort_values(by=['current_user_eval'], ascending=False)
sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)].sort_values(by=['non_user_eval'], ascending=False)
texts_to_label_ac = sorted_probs_current[~sorted_probs_current.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'alcohol_answer_label']].drop_duplicates().head(100)
texts_to_label_ac = texts_to_label_ac.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_ac.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label.processed_text))][['text', 'processed_text', 'alcohol_answer_label']].drop_duplicates().head(100))
texts_to_label_ac = texts_to_label_ac.merge(smoking_corpus.rename(columns={'label': 'roken_answer_label'})[['processed_text', 'roken_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_ac = texts_to_label_ac.merge(drugs_corpus.rename(columns={'label': 'drugs_answer_label'})[['processed_text', 'drugs_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_merged = texts_to_label.append(texts_to_label_ac, ignore_index = True)
texts_to_label_merged = texts_to_label_merged.drop_duplicates()

/tmp/ipykernel_4182246/1167704707.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_ac = texts_to_label_ac.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_ac.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label.processed_text))][['text', 'processed_text', 'alcohol_answer_label']].drop_duplicates().head(100))
/tmp/ipykernel_4182246/1167704707.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_merged = texts_to_label.append(texts_to_label_ac, ignore_index = True)


In [36]:
texts_to_label_merged

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Samenvatting: \tVoorgeschiedenis: pleiomorf ad...,samenvatting: voorgeschiedenis: pleiomorf ade...,Rookt,Niets gevonden,Niets gevonden
1,Anamnese: \t Verkort consult: Voorgeschiedenis...,anamnese: verkort consult: voorgeschiedenis:...,Rookt,Ja,Nee
2,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
3,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
4,Anamnese: \t Verkort consult: TC: Aandachtspun...,anamnese: verkort consult: tc: aandachtspunt...,Rookt,Nee,Nee
...,...,...,...,...,...
495,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
496,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
497,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
498,Reden van komst / Verwijzing: \tReden verwijzi...,reden van komst / verwijzing: reden verwijzin...,Rookt niet,Nee,Nee


# Drugs

In [37]:
predicted_nb_df = drugs_model.decision_function(drugs_x_test)
predicted_nb = drugs_model.predict(drugs_x_test)
x_test_probs = pd.DataFrame(drugs_x_test, columns=['processed_text'])
x_test_probs['current_user_eval'] = [x[0] for x in predicted_nb_df]
x_test_probs['non_user_eval'] = [x[1] for x in predicted_nb_df]
x_test_probs['nothing_found_eval'] = [x[2] for x in predicted_nb_df]
x_test_probs['predicted_label'] = predicted_nb
x_test_probs['drugs_answer_label'] = drugs_y_test
x_test_probs = x_test_probs.merge(drugs_corpus[['text', 'processed_text']], left_on='processed_text', right_on='processed_text', how='left')

In [38]:
sorted_probs_current = x_test_probs.sort_values(by=['current_user_eval'], ascending=False)
sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)].sort_values(by=['non_user_eval'], ascending=False)
texts_to_label_dr = sorted_probs_current[['text', 'processed_text', 'drugs_answer_label']][~sorted_probs_current.processed_text.isin(texts_to_label_merged.processed_text)].head(100)
texts_to_label_dr = texts_to_label_dr.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_dr.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label_merged.processed_text))][['text', 'processed_text', 'drugs_answer_label']].drop_duplicates().head(100))
texts_to_label_dr = texts_to_label_dr.merge(smoking_corpus.rename(columns={'label': 'roken_answer_label'})[['processed_text', 'roken_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_dr = texts_to_label_dr.merge(drinking_corpus.rename(columns={'label': 'alcohol_answer_label'})[['processed_text', 'alcohol_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_merged_merged = texts_to_label_merged.append(texts_to_label_dr, ignore_index = True)
texts_to_label_merged_merged = texts_to_label_merged_merged.drop_duplicates()

/tmp/ipykernel_4182246/4194658346.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_dr = texts_to_label_dr.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_dr.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label_merged.processed_text))][['text', 'processed_text', 'drugs_answer_label']].drop_duplicates().head(100))
/tmp/ipykernel_4182246/4194658346.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_merged_merged = texts_to_label_merged.append(texts_to_label_dr, ignore_index = True)


In [39]:
texts_to_label_merged_merged

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Samenvatting: \tVoorgeschiedenis: pleiomorf ad...,samenvatting: voorgeschiedenis: pleiomorf ade...,Rookt,Niets gevonden,Niets gevonden
1,Anamnese: \t Verkort consult: Voorgeschiedenis...,anamnese: verkort consult: voorgeschiedenis:...,Rookt,Ja,Nee
2,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
3,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
4,Anamnese: \t Verkort consult: TC: Aandachtspun...,anamnese: verkort consult: tc: aandachtspunt...,Rookt,Nee,Nee
...,...,...,...,...,...
695,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
696,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
697,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Ja,Nee
698,Anamnese: \t Verkort consult: tc op 25-10-- ge...,anamnese: verkort consult: tc op 25-10-- gee...,Rookte,Ja,Nee


In [40]:
texts_to_label = texts_to_label_merged_merged.drop_duplicates()

In [41]:
texts_to_label

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Samenvatting: \tVoorgeschiedenis: pleiomorf ad...,samenvatting: voorgeschiedenis: pleiomorf ade...,Rookt,Niets gevonden,Niets gevonden
1,Anamnese: \t Verkort consult: Voorgeschiedenis...,anamnese: verkort consult: voorgeschiedenis:...,Rookt,Ja,Nee
2,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
3,Samenvatting: \tVoorgeschiedenis: geen bijzond...,samenvatting: voorgeschiedenis: geen bijzonde...,Rookt,Niets gevonden,Niets gevonden
4,Anamnese: \t Verkort consult: TC: Aandachtspun...,anamnese: verkort consult: tc: aandachtspunt...,Rookt,Nee,Nee
...,...,...,...,...,...
695,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
696,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
697,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Ja,Nee
698,Anamnese: \t Verkort consult: tc op 25-10-- ge...,anamnese: verkort consult: tc op 25-10-- gee...,Rookte,Ja,Nee


In [42]:
full_labels_full

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,Niets gevonden,Niets gevonden
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,Niets gevonden,Niets gevonden
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,Niets gevonden,Niets gevonden
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,Niets gevonden,Niets gevonden
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,Ja,Nee
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,Niets gevonden,Niets gevonden
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,Niets gevonden,Niets gevonden


In [43]:
full_labels_no_dup = full_labels_full[~full_labels_full.content.isin(texts_to_label.text)]

In [44]:
full_labels_no_dup

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,Niets gevonden,Niets gevonden
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,Niets gevonden,Niets gevonden
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,Niets gevonden,Niets gevonden
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,Niets gevonden,Niets gevonden
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,Ja,Nee
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,Niets gevonden,Niets gevonden
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,Niets gevonden,Niets gevonden


In [45]:
random_samples = full_labels_no_dup.sample(frac=1).head(1100 - len(texts_to_label.index))

In [46]:
full_samples = add_processed_text(random_samples, 'content')

In [47]:
full_samples

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label,processed_text
10907,399992,UITSLAG MICROBIOLOGIE\t\t[INITIALS][LASTNAME] ...,Niets gevonden,Niets gevonden,Niets gevonden,uitslag microbiologie [initials][lastname] ...
57710,647043,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden,"geachte collega, bovengenoemde patiente was o..."
8024,370035,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Ja,Ja,anamnese: verkort consult: aandachtspunten u...
16860,435711,"Geachte collega,\t\tVoor bovengenoemde patiënt...",Niets gevonden,Niets gevonden,Niets gevonden,"geachte collega, voor bovengenoemde patient h..."
16690,434872,"Geachte collega,\t\tReden van komst: premature...",Niets gevonden,Niets gevonden,Niets gevonden,"geachte collega, reden van komst: premature c..."
...,...,...,...,...,...,...
32469,555092,Anamnese: \tVerwijzer: Huisarts\tReden van kom...,Niets gevonden,Niets gevonden,Niets gevonden,anamnese: verwijzer: huisarts reden van komst...
11041,401515,"Zeer geachte collega,\t\tBovengenoemde patiënt...",Niets gevonden,Niets gevonden,Niets gevonden,"zeer geachte collega, bovengenoemde patient z..."
59080,650370,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Nee,Nee,anamnese: verkort consult: aandachtspunten u...
8736,382740,1-0-CTCUEBeloop: \tBeloop: Huisarts ingelicht,Niets gevonden,Niets gevonden,Niets gevonden,1-0-ctcuebeloop: beloop: huisarts ingelicht


In [27]:
text	processed_text	roken_answer_label	alcohol_answer_label	drugs_answer_label

SyntaxError: invalid syntax (323927493.py, line 1)

In [48]:
full_texts_to_label = texts_to_label[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']].append(full_samples.rename(columns={'content':'text'})[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']])

/tmp/ipykernel_4182246/923319018.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_texts_to_label = texts_to_label[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']].append(full_samples.rename(columns={'content':'text'})[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']])


In [49]:
full_texts_to_label

,text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Samenvatting: \tVoorgeschiedenis: pleiomorf ad...,Rookt,Niets gevonden,Niets gevonden
1,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookt,Ja,Nee
2,Samenvatting: \tVoorgeschiedenis: geen bijzond...,Rookt,Niets gevonden,Niets gevonden
3,Samenvatting: \tVoorgeschiedenis: geen bijzond...,Rookt,Niets gevonden,Niets gevonden
4,Anamnese: \t Verkort consult: TC: Aandachtspun...,Rookt,Nee,Nee
...,...,...,...,...
32469,Anamnese: \tVerwijzer: Huisarts\tReden van kom...,Niets gevonden,Niets gevonden,Niets gevonden
11041,"Zeer geachte collega,\t\tBovengenoemde patiënt...",Niets gevonden,Niets gevonden,Niets gevonden
59080,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Nee,Nee
8736,1-0-CTCUEBeloop: \tBeloop: Huisarts ingelicht,Niets gevonden,Niets gevonden,Niets gevonden


In [50]:
full_texts_to_label.to_csv('full_texts_to_label_top.csv')

In [30]:
full_labels_no_dup.sample(frac=1).head(1)

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
56849,644975,"Samenvatting: \tElectief, 3VD \t\t06-05-2022 C...",Niets gevonden,Niets gevonden,Niets gevonden


In [51]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [52]:
full_labels_no_dup.sample(frac=1).head(3)

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
45460,610571,"Beloop: \tBeloop: Cardiale voorgeschiedenis:\t\t\t2007 anterolateraal myocardinfarct waarvoor primaire PCI RDA. Reststenose in CX en IM waarvoor medicamenteus beleid\t2008 ACS daarbij diffuus 3VD onaantrekkelijk voor revascularisatie vervolgens medicamenteus beleid. \t2014 Hartinfarct in Turkije; tweemaal PCI\t2014 AP functionele klasse II/IV CCS; daarnaast ook atypische POB.\t2014 Redelijke systolische functie met apico-septale akinesie.\t2018: laatste controle dr. Schotborgh toen 1 jaar afgesproken\t\tOverige voorgeschiedenis:\tCOPD\tHypertensie\t4-2-20 tot 6-2-20 opname ivm pneumonie linker onderkwab\tAllergisch astma, late onset met gefixeerde obstructieHypertensie\t\tMedicatie\tAscal 1dd 100 mg\tLipitor 1dd 40 mg\tNTG spray zn\tCoveram 1 dd 10/5\tPromocard 1 dd 60 mg\tPantozol 40 mg\tZaldiar\tFlutiform 250/10 2dd2\t\tAllergieën\tGeen bekend\t\tIntoxicaties\tRookt niet, geen alcohol, geen drugs\t\tAnamnese\t\t\tAanvullend onderzoek: \t\tLab 4-2-20:\tLaag NtproBNP en hsTNT\tNormale nierfunctie\t\t\t\tConclusie: \tConclusie: Patiente bekend met coronairlijden wordt poliklinisch gezien ivm:\t1. klachten?\t2. LDL 4.7 in 2017. \t\t\tBeleid: \tBeleid: Beleid\tVoorbereding:\tIndien klachtenvrij retour huisarts evt nieuwe verwijzing.",Rookt niet,Nee,Nee
20073,451432,"Geachte collega,\t\tMet bovengenoemde patiënte had ik op 21-12-2022 telefonisch contact op de Polikliniek Dermatologie na polibezoek op 13-12-2022.\t\tReden van verwijzing\ticc reuma: bcc rechter neusvleugel? \t\tConclusie \tPrimair nodulair groeiend basaalcelcarcinoom neusbrug rechts \t \tBeleid \tfoto + biopt \tuitleg over basaalcelcarcinomen\tuitleg over mohs en over alternatief conventionele excisie \tMohs chirurgie wordt ingepland\t\tAnamnese\tAnamnese door coassistent Lowieke Snorn\tHuidafwijking sinds 1,5 jaar. Komt vaak een bloedkorst op, geneest niet.\tZit weinig in de zon\tDermatologische VG: Naevi naecocellularis en Verruca seborrhoica (2018 neusrug links, toen currettage) \tFA: negatief voor huidkanker en negatief voor melanomen.\tall: pleisters+, med-\tpm-/icd-\tMed: foliumzuur, hydroxychloroQ, calc chew, esomeprazol\t \tLichamelijk onderzoek \tIn toto inspectie van de huid: \tNeusbrug rechts 1 cm grote glanzende deels geel doorschemerende papel met crust \t\t\tAanvullend onderzoek \tHuidbiopt neusbrug rechts: nodulair groeiend basaalcelcarcinoom. De invasiediepte kan niet worden bepaald door fragmentatie, maar dit lijkt tenminste tot ruim in de reticulaire dermis.\t \t\t\t\t\t\tMet collegiale hoogachting,\t\tdr. M.S. van Kester, dermatoloog\tAGB code: 03307974",Niets gevonden,Niets gevonden,Niets gevonden
28256,535844,"Samenvatting: \tMevr belde op naar de poli. Heeft de afspraak van 09-01-2019 bij Dr.Quarles geannuleerd, mevr gaat over naar een ander zkh.",Niets gevonden,Niets gevonden,Niets gevonden
